In [1]:
# Instala pacotes necessários em silêncio
%pip install -q imbalanced-learn pandas scikit-learn seaborn matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Importação das Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from imblearn.over_sampling import SMOTE, ADASYN

**Análise Exploratória dos dados **

In [ ]:
# Configurações de visualização
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Carregamento dos dados
url = "https://raw.githubusercontent.com/pcbrom/perceptron-mlp-cnn/refs/heads/main/data/diabetes.csv"
df = pd.read_csv(url)

print("Dimensão do dataset:", df.shape)
print("\nPrimeiras linhas do dataset:")
print(df.head())

print("\nInformações gerais:")
print(df.info())

print("\nEstatísticas descritivas:")
print(df.describe())

print("\nValores ausentes:")
print(df.isnull().sum())

# Algumas colunas usam 0 como valor ausente
zero_cols = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
print("\nValores Nulos Por Colunas:")
print((df[zero_cols] == 0).sum())

# Histograma das variáveis
df.hist(bins=30, figsize=(15, 10), color="steelblue")
plt.suptitle("Distribuição das Variáveis")
plt.show()

# Boxplots para identificar outliers
for col in df.columns[:-1]:
    sns.boxplot(x=df[col])
    plt.title(f"Boxplot de {col}")
    plt.show()

# Matriz de correlação
corr_matrix = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriz de Correlação")
plt.show()

# Distribuição das classes
sns.countplot(x="Outcome", data=df, palette="viridis")
plt.title("Distribuição das Classes (Outcome)")
plt.xticks([0, 1], ["Não Diabético", "Diabético"])
plt.xlabel("Classe")
plt.ylabel("Frequência")
plt.show()

# Comparação entre classes para variáveis principais
features_to_plot = ["Glucose", "BMI", "Age"]
for feature in features_to_plot:
    sns.histplot(data=df, x=feature, hue="Outcome", kde=True, element="step")
    plt.title(f"Distribuição de {feature} por Classe")
    plt.show()

**Regressão Logística Sem a Utilização de SMOTE e ADASYN**

SMOTE e ADASYN são métodos de oversampling supervisionado usados antes do treinamento do modelo para balancear datasets desbalanceados.

SMOTE (Synthetic Minority Over-sampling Technique) gera novas amostras sintéticas da classe minoritária.As novas amostras são criadas interpolando entre amostras reais e seus vizinhos mais próximos.Isso evita simplesmente duplicar dados existentes, o que poderia causar overfitting. No dataset em quentão a classe diabéticos é menor que a classe não diabéticos.

**Regressão Logística SEM Utilização de SMOTE e ADASYN. Com Diversos Limiares**

In [ ]:
# Substitui zeros por NaN nas colunas relevantes
cols_with_zeros = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
df[cols_with_zeros] = df[cols_with_zeros].replace(0, np.nan)

# Imputação dos valores ausentes com a mediana
imputer = SimpleImputer(strategy="median")
df[cols_with_zeros] = imputer.fit_transform(df[cols_with_zeros])

# Separação de features e target
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

# Padronização dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisão em treino e teste (estratificada)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Treinamento do modelo de regressão logística
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Probabilidades previstas
y_proba = model.predict_proba(X_test)[:, 1]

# Avaliação para múltiplos limiares
thresholds = np.arange(0.1, 0.91, 0.05)
resultados = []

for thresh in thresholds:
    y_pred_thresh = (y_proba >= thresh).astype(int)
    precision = precision_score(y_test, y_pred_thresh)
    recall = recall_score(y_test, y_pred_thresh)
    f1 = f1_score(y_test, y_pred_thresh)
    resultados.append((thresh, precision, recall, f1))

df_limiar = pd.DataFrame(
    resultados, columns=["Limiar", "Precisão", "Recall", "F1-Score"]
)
print("\nAvaliação por múltiplos limiares:")
print(df_limiar)

# Seleciona o melhor limiar com base no F1-score
melhor_limiar = df_limiar.loc[df_limiar["F1-Score"].idxmax(), "Limiar"]
print(f"\nMelhor limiar com base no F1-score: {melhor_limiar:.2f}")

# Predição final com limiar otimizado
y_pred_final = (y_proba >= melhor_limiar).astype(int)

print("\nAvaliação Final com Limiar Otimizado:")
print("Acurácia:", accuracy_score(y_test, y_pred_final))
print("Precisão:", precision_score(y_test, y_pred_final))
print("Revocação (Recall):", recall_score(y_test, y_pred_final))
print("F1-score:", f1_score(y_test, y_pred_final))

# Plot das métricas por limiar
plt.figure(figsize=(10, 6))
plt.plot(df_limiar["Limiar"], df_limiar["Precisão"], label="Precisão")
plt.plot(df_limiar["Limiar"], df_limiar["Recall"], label="Recall")
plt.plot(df_limiar["Limiar"], df_limiar["F1-Score"], label="F1-Score", linewidth=2)
plt.xlabel("Limiar")
plt.ylabel("Métrica")
plt.title("Avaliação de Métricas por Limiar")
plt.legend()
plt.grid(True)
plt.show()

# Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"Curva ROC (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="gray", linestyle="--")
plt.xlabel("Taxa de Falsos Positivos (FPR)")
plt.ylabel("Taxa de Verdadeiros Positivos (TPR)")
plt.title("Curva ROC - Regressão Logística")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

**Regressão Logística Com a Utilização de SMOTE e Diversos Limiares**

In [ ]:
# Imputação novamente (já foi feita antes, pode ser removida)
imputer = SimpleImputer(strategy="median")
df[cols_with_zeros] = imputer.fit_transform(df[cols_with_zeros])

# Separação de features e target
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

# Padronização dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisão em treino e teste (estratificada)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

# SMOTE para balanceamento da base de treino
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

# ADASYN para balanceamento da base de treino
adasyn = ADASYN(random_state=42)
X_adasyn, y_adasyn = adasyn.fit_resample(X_train, y_train)

model = LogisticRegression(max_iter=1000)

# Modelo com SMOTE
model.fit(X_smote, y_smote)

# Probabilidades previstas
y_proba_smote = model.predict_proba(X_test)[:, 1]
limiar = 0.9
y_pred_smote = (y_proba_smote >= limiar).astype(int)

print("\nAvaliação com SMOTE:")
print(
    classification_report(
        y_test, y_pred_smote, target_names=["Não Diabético", "Diabético"]
    )
)

# Avaliação multi-limiar para modelo com SMOTE
thresholds = np.arange(0.1, 0.91, 0.05)
resultados = []

for thresh in thresholds:
    y_pred_thresh = (y_proba_smote >= thresh).astype(int)
    precision = precision_score(y_test, y_pred_thresh)
    recall = recall_score(y_test, y_pred_thresh)
    f1 = f1_score(y_test, y_pred_thresh)
    resultados.append((thresh, precision, recall, f1))

# DataFrame com métricas por limiar
df_limiar = pd.DataFrame(
    resultados, columns=["Limiar", "Precisão", "Recall", "F1-Score"]
)
print("\nAvaliação por múltiplos limiares (SMOTE):")
print(df_limiar)

# Plot das métricas por limiar
plt.figure(figsize=(10, 6))
plt.plot(df_limiar["Limiar"], df_limiar["Precisão"], label="Precisão")
plt.plot(df_limiar["Limiar"], df_limiar["Recall"], label="Recall")
plt.plot(df_limiar["Limiar"], df_limiar["F1-Score"], label="F1-Score", linewidth=2)
plt.xlabel("Limiar")
plt.ylabel("Métrica")
plt.title("Avaliação de Métricas por Limiar (SMOTE)")
plt.legend()
plt.grid(True)
plt.show()

# Curva ROC para SMOTE
fpr_smote, tpr_smote, _ = roc_curve(y_test, y_proba_smote)
auc_smote = auc(fpr_smote, tpr_smote)
plt.figure(figsize=(8, 6))
plt.plot(
    fpr_smote, tpr_smote, color="blue", lw=2, label=f"SMOTE (AUC = {auc_smote:.2f})"
)
plt.plot([0, 1], [0, 1], linestyle="--", color="gray", lw=1)
plt.xlabel("Taxa de Falsos Positivos (FPR)")
plt.ylabel("Taxa de Verdadeiros Positivos (TPR)")
plt.title("Curva ROC - Regressão Logística com SMOTE")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

**Regressão Logística Com a Utilização de ADASYN e Diversos Limiares**

In [ ]:
# Modelo com ADASYN
model.fit(X_adasyn, y_adasyn)
y_proba_adasyn = model.predict_proba(X_test)[:, 1]

# Avaliação multi-limiar automática
thresholds = np.arange(0.1, 0.91, 0.05)
resultados_adasyn = []

for thresh in thresholds:
    y_pred_thresh = (y_proba_adasyn >= thresh).astype(int)
    precision = precision_score(y_test, y_pred_thresh)
    recall = recall_score(y_test, y_pred_thresh)
    f1 = f1_score(y_test, y_pred_thresh)
    resultados_adasyn.append((thresh, precision, recall, f1))

# DataFrame de resultados
df_adasyn = pd.DataFrame(
    resultados_adasyn, columns=["Limiar", "Precisão", "Recall", "F1-Score"]
)
print("\nAvaliação por múltiplos limiares (ADASYN):")
print(df_adasyn)

# Limiar ótimo com base no F1-score
melhor_limiar_adasyn = df_adasyn.loc[df_adasyn["F1-Score"].idxmax(), "Limiar"]
print(f"\nMelhor limiar para ADASYN (F1-score): {melhor_limiar_adasyn:.2f}")

# Previsão com limiar ótimo
y_pred_adasyn = (y_proba_adasyn >= melhor_limiar_adasyn).astype(int)

# Relatório final
print("\nAvaliação final com ADASYN (limiar otimizado):")
print(
    classification_report(
        y_test, y_pred_adasyn, target_names=["Não Diabético", "Diabético"]
    )
)

# Plot das métricas por limiar
plt.figure(figsize=(10, 6))
plt.plot(df_adasyn["Limiar"], df_adasyn["Precisão"], label="Precisão")
plt.plot(df_adasyn["Limiar"], df_adasyn["Recall"], label="Recall")
plt.plot(df_adasyn["Limiar"], df_adasyn["F1-Score"], label="F1-Score", linewidth=2)
plt.xlabel("Limiar")
plt.ylabel("Métrica")
plt.title("Avaliação de Métricas por Limiar (ADASYN)")
plt.legend()
plt.grid(True)
plt.show()

# Curva ROC para ADASYN
fpr_adasyn, tpr_adasyn, _ = roc_curve(y_test, y_proba_adasyn)
auc_adasyn = auc(fpr_adasyn, tpr_adasyn)
plt.figure(figsize=(8, 6))
plt.plot(
    fpr_adasyn,
    tpr_adasyn,
    color="darkorange",
    lw=2,
    label=f"ADASYN (AUC = {auc_adasyn:.2f})",
)
plt.plot([0, 1], [0, 1], linestyle="--", color="gray", lw=1)
plt.xlabel("Taxa de Falsos Positivos (FPR)")
plt.ylabel("Taxa de Verdadeiros Positivos (TPR)")
plt.title("Curva ROC - Regressão Logística com ADASYN")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

Comparação Curva ROC

In [ ]:
# Curvas ROC para todos os modelos
fpr_orig, tpr_orig, _ = roc_curve(y_test, y_proba)
fpr_smote, tpr_smote, _ = roc_curve(y_test, y_proba_smote)
fpr_adasyn, tpr_adasyn, _ = roc_curve(y_test, y_proba_adasyn)

auc_orig = auc(fpr_orig, tpr_orig)
auc_smote = auc(fpr_smote, tpr_smote)
auc_adasyn = auc(fpr_adasyn, tpr_adasyn)

plt.figure(figsize=(10, 6))
plt.plot(fpr_orig, tpr_orig, label=f"Original (AUC = {auc_orig:.2f})", lw=2)
plt.plot(fpr_smote, tpr_smote, label=f"SMOTE    (AUC = {auc_smote:.2f})", lw=2)
plt.plot(fpr_adasyn, tpr_adasyn, label=f"ADASYN   (AUC = {auc_adasyn:.2f})", lw=2)

plt.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Aleatório")

plt.xlabel("Taxa de Falsos Positivos (FPR)")
plt.ylabel("Taxa de Verdadeiros Positivos (TPR)")
plt.title("Curva ROC - Comparação: Original, SMOTE e ADASYN")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

# Comparação de F1-score e Recall para limiar 0.5
y_pred_orig = (y_proba >= 0.5).astype(int)
y_pred_smote = (y_proba_smote >= 0.5).astype(int)
y_pred_adasyn = (y_proba_adasyn >= 0.5).astype(int)

f1_orig = f1_score(y_test, y_pred_orig)
f1_smote = f1_score(y_test, y_pred_smote)
f1_adasyn = f1_score(y_test, y_pred_adasyn)

recall_orig = recall_score(y_test, y_pred_orig)
recall_smote = recall_score(y_test, y_pred_smote)
recall_adasyn = recall_score(y_test, y_pred_adasyn)

print("\nComparação de F1-score e Recall:")
print(f"Original  → F1: {f1_orig:.2f} | Recall: {recall_orig:.2f}")
print(f"SMOTE     → F1: {f1_smote:.2f} | Recall: {recall_smote:.2f}")
print(f"ADASYN    → F1: {f1_adasyn:.2f} | Recall: {recall_adasyn:.2f}")

# Gráfico comparativo de F1-score e Recall
labels = ["Original", "SMOTE", "ADASYN"]
f1_scores = [f1_orig, f1_smote, f1_adasyn]
recalls = [recall_orig, recall_smote, recall_adasyn]

x = np.arange(len(labels))
width = 0.35

plt.figure(figsize=(8, 5))
plt.bar(x - width / 2, f1_scores, width, label="F1-Score")
plt.bar(x + width / 2, recalls, width, label="Recall")
plt.xticks(x, labels)
plt.ylabel("Valor")
plt.title("Comparação de F1-Score e Recall")
plt.ylim(0, 1)
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()